In [ ]:
import math as m
import random

import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Rectangle

import ipywidgets as widgets

In [ ]:
EXT_SIZE_X, EXT_SIZE_Y = 80, 60

INT_OFFS_X, INT_OFFS_Y = 20, 20
INT_SIZE_X, INT_SIZE_Y = 20, 20

EXT_BORDER = +300.0
INT_BORDER = +320.0

STEP = 1

def func(x, z):
    # return 30
    f0, beta = 60, 0.1
    x0, z0 = 45, 9
    return f0 * m.exp(-beta * ((x - x0)**2 + (z - z0)**2))

K = 20

In [ ]:
ibx0, iby0 = INT_OFFS_X, INT_OFFS_Y
ibx1, iby1 = INT_OFFS_X + INT_SIZE_X, INT_OFFS_Y + INT_SIZE_Y

def is_outside(y, x):
    if x <= 0 or x >= EXT_SIZE_X or y <= 0 or y >= EXT_SIZE_Y:
        return True
    if ibx0 <= x <= ibx1 and iby0 <= y <= iby1:
        return True
    return False

def dist_pt_segm_hor(ptx: int, pty: int, sx1: int, sx2: int, sy: int):
    assert sx1 < sx2
    if ptx < sx1:
        return m.hypot(sx1 - ptx, abs(pty - sy))
    elif ptx > sx2:
        return m.hypot(ptx - sx2, abs(pty - sy))
    else:
        return abs(pty - sy)

def dist_pt_segm_vert(ptx: int, pty: int, sy1: int, sy2: int, sx: int):
    return dist_pt_segm_hor(pty, ptx, sy1, sy2, sx)

# Assumed that point is valid
def dist_to_border(y: int, x: int):
    # TODO: dont check all borders
    # if x < INT_OFFS_X:
    #     res = min(res, dist_pt_segm_vert(x, y, ))

    min_ext = min([
        dist_pt_segm_vert(x, y, 0, EXT_SIZE_Y, 0), # Ext left
        dist_pt_segm_vert(x, y, 0, EXT_SIZE_Y, EXT_SIZE_X), # Ext right

        dist_pt_segm_hor(x, y, 0, EXT_SIZE_X, 0), # Ext top
        dist_pt_segm_hor(x, y, 0, EXT_SIZE_X, EXT_SIZE_Y), # Ext bottom
    ])

    min_int = min([
        dist_pt_segm_vert(x, y, iby0, iby1, ibx0), # Int left
        dist_pt_segm_vert(x, y, iby0, iby1, ibx1), # Int right

        dist_pt_segm_hor(x, y, ibx0, ibx1, iby0), # Int top
        dist_pt_segm_hor(x, y, ibx0, ibx1, iby1), # Int bottom
    ])

    if min_ext < min_int:
        return min_ext, EXT_BORDER  # TODO: Non constant temperature
    else:
        return min_int, INT_BORDER

In [ ]:
def FUNC2(x, y): return func(y, x) / K  # 45 9

Simpson's integration:
$
I^* = \frac{h}{6} \sum_i \left(f_i + 4f_{i+0.5} + f_{i+1}\right)
$

In [ ]:
INTEGR_STEP = 1e-2

def integrate_segm(x1: int, y1: int, x2: int, y2: int):
    dx, dy = abs(x2 - x1), abs(y2 - y1)
    segm_len = m.hypot(dx, dy)
    halfx = INTEGR_STEP / 2.0 * dx
    halfy = INTEGR_STEP / 2.0 * dy

    s = 0
    ptx, pty = x1, y1
    lastval = FUNC2(ptx, pty)

    for i in range(int(segm_len / INTEGR_STEP)):
        # s += FUNC2(ptx, pty)
        s += lastval

        s += 4 * FUNC2(ptx + halfx, pty + halfy)

        ptx += INTEGR_STEP
        pty += INTEGR_STEP
        
        # s += FUNC2(ptx, pty)
        lastval = FUNC2(ptx, pty)
        s += lastval

    res = s * INTEGR_STEP / 6.0
    return res

In [ ]:
# Monte-carlo method (floating)

ITERS_COUNT = 50
EPS = 1e-2

# Set fixed step for using semi-floating walk
# FIXED_STEP = None
FIXED_STEP = 1

def prob_trace(y: float, x: float):
    _2_PI = 2 * m.pi

    hops = 0
    res = 0

    dist, btemp = dist_to_border(y, x)
    while dist > EPS and not is_outside(y, x):
        angle = _2_PI * random.random()
        if FIXED_STEP is None:
            nx = x + dist * m.cos(angle)
            ny = y + dist * m.sin(angle)
        else:
            nx = x + FIXED_STEP * m.cos(angle)
            ny = y + FIXED_STEP * m.sin(angle)

        res += integrate_segm(x, y, nx, ny) * 0.25
        x, y = nx, ny

        dist, btemp = dist_to_border(y, x)
        hops += 1
    
    res += btemp
    return res

def prob_point(j, i):
    res = 0
    for _ in range(ITERS_COUNT):
        res += prob_trace(j, i)
    res /= ITERS_COUNT
    return res

In [ ]:
TARGET = (45, 9)
res = prob_point(*TARGET)
print('Res: {:.3f}'.format(res))
orig = 330.6864168849065
print('Err: {:.3f}%'.format((abs(res - orig) / orig) * 100))